# Day 4: Exploratory Data Analysis I - Vels University Student Data Analysis

This notebook covers advanced GroupBy operations and filtering techniques in pandas using Vels University student data.

*Created by: MysticDevil and Nandhan K*

## 1. Setting up the Environment and Sample Data

In [ ]:
import pandas as pd
import numpy as np
# Create sample student data
np.random.seed(42)
# Generate sample data
departments = ['Computer Science', 'Information Technology', 'AI & DS', 'Cyber Security']
semesters = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th']
data = {
    'date': pd.date_range('2023-01-01', '2023-12-31', freq='D'),
    'roll_number': np.arange(1, 366),
    'department': np.random.choice(departments, 365),
    'semester': np.random.choice(semesters, 365),
    'attendance': np.random.uniform(65, 100, 365),
    'marks': np.random.normal(75, 15, 365),
    'student_type': np.random.choice(['Regular', 'Lateral Entry', 'Transfer'], 365)
}
# Ensure marks are between 0 and 100
data['marks'] = np.clip(data['marks'], 0, 100)

df = pd.DataFrame(data)
print("Sample of the Vels University student dataset:")
print(df.head())
df.to_excel("vels_university_students.xlsx", index=False)

Sample of the Vels University student dataset:
        date  roll_number        department semester  attendance       marks  \
0 2023-01-01            1           AI & DS      7th   78.513592   80.916782   
1 2023-01-02            2    Cyber Security      6th   98.641670   68.685233   
2 2023-01-03            3  Computer Science      2nd   96.687272   79.346623   
3 2023-01-04            4           AI & DS      3rd   71.852690  100.000000   
4 2023-01-05            5           AI & DS      7th   67.427646   88.066871   

    student_type  
0        Regular  
1  Lateral Entry  
2  Lateral Entry  
3       Transfer  
4  Lateral Entry  


## 2. Basic GroupBy Operations

In [ ]:
# Group by single column - Department-wise performance
dept_stats = df.groupby('department')['marks'].agg(['mean', 'min', 'max', 'count'])
print("Academic performance by department:")
print(dept_stats.round(2))

# Group by multiple columns - Department and semester wise analysis
dept_sem_stats = df.groupby(['department', 'semester'])['marks'].mean().unstack()
print("\nAverage marks by department and semester:")
print(dept_sem_stats.round(2))

## 3. Advanced GroupBy Operations

In [ ]:
# Custom aggregation function for student performance
def student_metrics(x):
    return pd.Series({
        'average_marks': x.mean(),
        'highest_mark': x.max(),
        'lowest_mark': x.min(),
        'pass_rate': (x >= 40).mean() * 100,  # Percentage of students passing
        'distinction_rate': (x >= 75).mean() * 100  # Percentage of distinctions
    })

# Apply custom aggregation
advanced_stats = df.groupby(['department', 'semester'])['marks'].apply(student_metrics)
print("Detailed academic performance metrics:")
print(advanced_stats.round(2))

## 4. Time-based Grouping

In [ ]:
# Add time components
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['day_of_week'] = df['date'].dt.day_name()

# Monthly attendance trends
monthly_attendance = df.groupby(['month', 'department'])['attendance'].mean().unstack()
print("Monthly attendance trends by department:")
print(monthly_attendance.round(2))

# Quarterly performance analysis
quarterly_analysis = df.groupby(['quarter', 'department']).agg({
    'marks': ['mean', 'std'],
    'attendance': 'mean'
}).round(2)
print("\nQuarterly academic performance analysis:")
print(quarterly_analysis)

## 5. Advanced Filtering

In [ ]:
# Using query() - Find high performers
high_performers = df.query('marks > 85 and attendance > 90')
print("High performing students (marks > 85 and attendance > 90):")
print(high_performers.head())

# Using .isin() - Analyze specific departments
tech_departments = df[df['department'].isin(['Computer Science', 'Information Technology'])]
print("\nPerformance in Technical Departments:")
print(tech_departments.groupby('department')['marks'].mean().round(2))

# Complex conditions - Find exceptional students
condition = (
    (df['marks'] > df['marks'].mean()) &
    (df['attendance'] > df['attendance'].median()) &
    (df['student_type'] == 'Regular')
)
exceptional_students = df[condition]
print("\nExceptional regular students:")
print(exceptional_students.head())

## 6. Window Functions and Rolling Calculations

In [ ]:
# Sort by date
df_sorted = df.sort_values('date')

# Calculate rolling average attendance (7-day window)
df_sorted['rolling_avg_attendance'] = df_sorted.groupby('department')['attendance'].transform(
    lambda x: x.rolling(window=7, min_periods=1).mean()
)

# Calculate cumulative average marks
df_sorted['cumulative_avg_marks'] = df_sorted.groupby('department')['marks'].transform(
    lambda x: x.expanding().mean()
)

print("Rolling averages and cumulative statistics:")
print(df_sorted[['date', 'department', 'marks', 'attendance', 
                 'rolling_avg_attendance', 'cumulative_avg_marks']].head(10))

## 7. Practical Exercise: Sales Analysis

In [ ]:
# Comprehensive student performance analysis
def academic_analysis(df):
    # 1. Overall performance metrics
    print("1. Overall Academic Performance Metrics:")
    overall_metrics = df.agg({
        'marks': ['count', 'mean', 'std', 'min', 'max'],
        'attendance': ['mean', 'min', 'max']
    })
    print(overall_metrics.round(2))
    
    # 2. Department ranking
    print("\n2. Department Performance Ranking:")
    dept_ranking = df.groupby('department').agg({
        'marks': 'mean',
        'attendance': 'mean'
    }).sort_values('marks', ascending=False)
    print(dept_ranking.round(2))
    
    # 3. Performance by student type
    print("\n3. Performance by Student Type:")
    student_type_perf = df.groupby(['department', 'student_type'])['marks'].mean().unstack()
    print(student_type_perf.round(2))
    
    # 4. Monthly progress
    print("\n4. Monthly Progress:")
    monthly_progress = df.groupby('month')[['marks', 'attendance']].mean()
    print(monthly_progress.round(2))

# Run the analysis
academic_analysis(df)